<a href="https://colab.research.google.com/github/dickylcd/Financial_Analysis/blob/main/VALE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Load the historical stock price data - connect from Yahoo Finace API
#!pip install yfinance


In [ ]:
'''
import yfinance
from datetime import datetime

# Define the dates of data
ten_year = datetime.today().year - 10
month = datetime.today().month
day = datetime.today().day

start_date = datetime.strptime(f"{ten_year}-{month}-{day}",'%Y-%m-%d').date()
end_date = datetime.today()

# Define stock symbol
stock_symbol = 'AVGO'

ultratech_df = yfinance.download(stock_symbol, start=start_date, end=end_date)
ultratech_df.columns
'''

[*********************100%%**********************]  1 of 1 completed


Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

# Load the historical stock price data - connect from Google Drive
from google.colab import drive
drive.mount('/content/drive')

#file_path = '/mnt/data/NASDAQ_ AVGO.csv'
file_path = '/content/drive/MyDrive/a-AI_In_Action/NYSE_VALE.csv'

# Load the data into a DataFrame
stock_data = pd.read_csv(file_path, parse_dates=['Date'])
stock_data = stock_data[(stock_data['Date'] >= '2024-05-01') & (stock_data['Date'] <= '2024-08-16')]
stock_data.set_index('Date', inplace=True)

# Calculate moving averages
stock_data['MA20'] = stock_data['Close'].rolling(window=20).mean()
stock_data['MA50'] = stock_data['Close'].rolling(window=50).mean()

# Display the first few rows to understand the structure of the data
stock_data.head()


Mounted at /content/drive


,Open,High,Low,Close,Volume,MA20,MA50
Date,,,,,,,
2024-05-01 16:00:00,12.19,12.38,12.10,12.20,14476200,NaN,NaN
2024-05-02 16:00:00,12.45,12.56,12.34,12.52,27528776,NaN,NaN
2024-05-03 16:00:00,12.65,12.67,12.45,12.61,23144400,NaN,NaN
2024-05-06 16:00:00,12.66,12.73,12.59,12.59,17105545,NaN,NaN
2024-05-07 16:00:00,12.80,12.88,12.69,12.69,24043516,NaN,NaN


In [3]:
# Generate signals
stock_data['Signal'] = 0
stock_data['Signal'][20:] = np.where(stock_data['MA20'][20:] > stock_data['MA50'][20:], 1, 0)  # Buy signal
stock_data['Position'] = stock_data['Signal'].diff()

# Calculate returns
stock_data['Market Return'] = stock_data['Close'].pct_change()
stock_data['Strategy Return'] = stock_data['Market Return'] * stock_data['Signal'].shift(1)

# Performance metrics
total_return = (stock_data['Strategy Return'].cumsum()[-1])
risk_free_rate = 0.01  # assuming 1% risk-free rate
sharpe_ratio = (stock_data['Strategy Return'].mean() - risk_free_rate) / stock_data['Strategy Return'].std()


<ipython-input-3-e97291712a83>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_data['Signal'][20:] = np.where(stock_data['MA20'][20:] > stock_data['MA50'][20:], 1, 0)  # Buy signal
<ipython-input-3-e97291712a83>:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  total_return = (stock_data['Strategy Return'].cumsum()[-1])
<ipython-input-3-e97291712a83>:13: RuntimeWarning: divide by zero encountered in scalar divide
  sharpe_ratio = (stock_data['Strategy Return'].mean() - risk_free_rate) / stock_data['Strategy Return'].std()


In [4]:
# Visualization
fig = go.Figure()
fig.add_trace(go.Scatter(x=stock_data.index, y=stock_data['Close'], mode='lines', name='Close Price'))
fig.add_trace(go.Scatter(x=stock_data.index, y=stock_data['MA20'], mode='lines', name='20-Day MA'))
fig.add_trace(go.Scatter(x=stock_data.index, y=stock_data['MA50'], mode='lines', name='50-Day MA'))

# Buy signals
buy_signals = stock_data[stock_data['Position'] == 1]
sell_signals = stock_data[stock_data['Position'] == -1]
fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals['Close'], mode='markers', name='Buy Signal', marker=dict(color='green', size=10)))
fig.add_trace(go.Scatter(x=sell_signals.index, y=sell_signals['Close'], mode='markers', name='Sell Signal', marker=dict(color='red', size=10)))

fig.update_layout(title='Momentum Trading Strategy for NYSE_VALE',
                  xaxis_title='Date',
                  yaxis_title='Price',
                  legend_title='Legend')
fig.show()

# Print performance metrics
print(f'Total Return: {total_return:.2%}')
print(f'Sharpe Ratio: {sharpe_ratio:.2f}')

Total Return: 0.00%
Sharpe Ratio: -inf
